In [1]:
from glob import glob
import os

basePath = 'highway/valid/label/'

valid_filename = []
for filepath in glob(os.path.join(basePath, '*.xml')):
    filename = os.path.basename(filepath)  ## 상위폴더 경로 제거
    filename = filename[:-4]  ## .xml 제거
    valid_filename.append(filename)

len(valid_filename)

51

In [2]:
import pandas as pd

df = pd.DataFrame(valid_filename)
df.columns = ['valid_folder']
df.head()

,valid_folder
0,Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5...
1,Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5...
2,Suwon_CH01_20200722_1430_WED_9m_NH_highway_TW5...
3,Suwon_CH01_20200722_1930_WED_9m_NH_highway_TW5...
4,Suwon_CH01_20201012_1723_MON_9m_RH_highway_TW5...


In [3]:
import numpy as np


def glob_files(path, file_type="*"):
    search_string = os.path.join(path, file_type)
    files = glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


WIDTH = 1
HEIGHT = 1

def load_labels(path):
  files = glob_files(path, "*.txt")

  Y_data = []
  for file in files:
    with open(file) as f:
      lines = f.readlines()

      boxes = []
      for line in lines:
        tokens = line.split()

        class_id = int(tokens[0])
        xc = float(tokens[1]) * WIDTH
        yc = float(tokens[2]) * HEIGHT
        width = float(tokens[3]) * WIDTH
        height = float(tokens[4]) * HEIGHT

        boxes.append(np.array([class_id, xc, yc, width, height]))
        # print(class_id, xc, yc, width, height)

      Y_data.append(np.array(boxes))
      # print(lines)
  return Y_data  ## 이미지마다 탐지 대상 개수가 달라서(행 개수가 달라서) 정해진 shape의 넘파이 어레이로 변환할 수 없다! 변환하려면 패딩 등이 필요.

Y_test_ex = load_labels(os.path.join(basePath, 'Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD'))

In [4]:
len(Y_test_ex)

150

In [5]:
df_ex = pd.DataFrame(pd.DataFrame(Y_test_ex[0])[0].value_counts()).T
df_ex

,0.0,2.0,1.0
count,21,8,3


In [6]:
df_ex.loc['count', 0.0]

21

In [7]:
df.loc[0, 'car_cnt'] = df_ex.loc['count', 0.0]
df.head()

,valid_folder,car_cnt
0,Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5...,21.0
1,Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5...,NaN
2,Suwon_CH01_20200722_1430_WED_9m_NH_highway_TW5...,NaN
3,Suwon_CH01_20200722_1930_WED_9m_NH_highway_TW5...,NaN
4,Suwon_CH01_20201012_1723_MON_9m_RH_highway_TW5...,NaN


In [8]:
classes = ['car', 'bus', 'truck']

In [12]:
for i in range(3):
    # print(float(i))
    col = classes[i] + '_cnt'
    # print(col)
    df.loc[0, col] = df_ex.loc['count', float(i)]

In [13]:
df.head()

,valid_folder,car_cnt,bus_cnt,truck_cnt
0,Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5...,21.0,3.0,8.0
1,Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5...,NaN,NaN,NaN
2,Suwon_CH01_20200722_1430_WED_9m_NH_highway_TW5...,NaN,NaN,NaN
3,Suwon_CH01_20200722_1930_WED_9m_NH_highway_TW5...,NaN,NaN,NaN
4,Suwon_CH01_20201012_1723_MON_9m_RH_highway_TW5...,NaN,NaN,NaN
